In [7]:
import logging
import os
logging.basicConfig(level=logging.ERROR)

In [14]:
from datetime import datetime, timedelta
from typing import List
from termcolor import colored


from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain.experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

In [ ]:
var_name = ''
os.environ['OPENAI_API_KEY'] = var_name

In [16]:
USER_NAME = "Person A"  # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

In [18]:
import math
import faiss


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [19]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=tommies_memory,
)

In [20]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie's core characteristics include being confident, determined, and disciplined.


In [21]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [22]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is a person who has fond memories of his childhood dog, Bruno. He is currently in a new home and feeling hungry. Tommie tries to get some rest but is tired from driving a long distance. The noise from the road at night disturbs him. Additionally, Tommie's new neighbors have a cat.


In [23]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [25]:
interview_agent(tommie, "What do you like to do?")

'Tommie said "I really enjoy design and being creative. I find it relaxing and fulfilling. How about you?"'

In [26]:
interview_agent(tommie, "What are you looking forward to doing today?")

'Tommie said "I\'m actually currently looking for a job, so I\'m really looking forward to finding some opportunities and hopefully starting a new chapter in my career. How about you? What are you looking forward to doing today?"'

In [27]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Well, I have to admit that I\'ve been feeling a bit anxious lately. I\'ve just moved to a new home, and everything feels so unfamiliar. Plus, I\'m currently looking for a job, so there\'s definitely some worry about that. But I\'m trying to stay positive and hopeful. How about you? What are you most worried about today?"'

In [28]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [29]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Tommie wakes up to the sound of a noisy construction site outside his window. Tommie covers his ears with a pillow to try and block out the noise.
Tommie gets out of bed and heads to the kitchen to make himself some coffee. Tommie starts making coffee.
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. Tommie frantically searches through the moving boxes to find the coffee filters.
Tommie finally finds the filters and makes himself a cup of coffee. Tommie takes a sip of the coffee and feels a little more awake.
The coffee tastes bitter, and Tommie regrets not buying a better brand. Tommie grimaces at the bitter taste and decides to make a note to buy a better brand of coffee in the future.
Tommie checks his email and sees that he has no job offers yet. Tommie sighs and feels a bit disappointed but remains hopeful for future job offers.
Tommie spends some time updating his resume and cover letter. Tommie feels a sense of accomplis

In [34]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "Well, it\'s been a bit of a rollercoaster, to be honest. I\'ve been feeling frustrated with my job search lately and it\'s been a bit tough. But I did have a nice break earlier when I went for a walk in the park, and I got to play frisbee for a bit, although I did end up getting hit in the face with it, ouch! Oh, and there was this adorable dog that came up to me and it really brightened my day. But overall, I\'m still hopeful and excited about finding job opportunities. How about you? How has your day been?"'

In [35]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "I actually love coffee! It\'s one of my favorite drinks. How about you, do you enjoy coffee?"'

In [36]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Oh, I had a dog named Bruno when I was a kid! He was a golden retriever and he was such a loyal and playful companion. I have so many fond memories of playing fetch and going on walks with him. Unfortunately, I don\'t have a dog right now, but I did have a really nice interaction with a dog earlier today. It reminded me of the joy and comfort that dogs can bring. How about you, did you have any pets growing up?"'

In [37]:
eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)


eve = GenerativeAgent(
    name="Eve",
    age=34,
    traits="curious, helpful",  # You can add more persistent traits here
    status="N/A",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    daily_summaries=[
        (
            "Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie."
        )
    ],
    memory=eves_memory,
    verbose=False,
)

In [38]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_observations = [
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
]
for observation in eve_observations:
    eve.memory.add_memory(observation)

In [39]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and active individual who enjoys engaging in physical activities like playing tennis with her friend Xu. She is attentive to her surroundings and overhears conversations, such as her colleague mentioning that Tommie is difficult to work with. Additionally, Eve starts her day by waking up to an alarm.


In [40]:
interview_agent(eve, "How are you feeling about today?")

'Eve said "I\'m feeling pretty good about today! I had a great time playing tennis with Xu earlier, and I helped a coworker with a task at work. How about you? How are you feeling about today?"'

In [41]:
interview_agent(eve, "What do you know about Tommie?")

'Eve said "I\'ve heard that Tommie can be difficult to work with. Have you had any experiences with them?"'

In [42]:
interview_agent(
    eve,
    "Tommie is looking to find a job. What are are some things you'd like to ask him?",
)

'Eve said "Oh, I\'ve actually heard that Tommie can be difficult to work with. Have you had any experiences with them?"'

In [43]:
interview_agent(
    eve,
    "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Eve said "I understand. I\'ll do my best to keep the conversation going and ask Tommie as many questions as I can. Thank you for letting me know. Is there anything specific you would like me to ask him?"'

In [51]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [52]:
agents = [tommie, eve]
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?",
)

InvalidRequestError: This model's maximum context length is 4097 tokens. However, you requested 4121 tokens (2621 in the messages, 1500 in the completion). Please reduce the length of the messages or completion.

In [46]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is hopeful, excited, and motivated in his job search. He is grateful for words of encouragement and feels a sense of accomplishment when updating his resume. However, he can also feel frustrated and disappointed at times. He is curious and hopeful about job openings and actively seeks opportunities. Tommie also has a strong connection to his past, remembering his childhood dog, Bruno. He values his friendships and seeks support when feeling frustrated. Additionally, Tommie is playful and enjoys activities like playing frisbee.


In [47]:
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and supportive coworker who is willing to assist others with tasks. She enjoys playing tennis with her friend Xu and starts her day by eating porridge and waking up to an alarm. Eve is attentive and observant, as she overhears her colleague discussing Tommie being difficult to work with. She shows empathy and understanding when talking to Tommie, expressing her support and interest in his exploration of opportunities in design. Eve also demonstrates good communication skills by asking open-ended questions and actively engaging in conversations. Overall, Eve is open-minded, positive, and willing to learn new things.


In [48]:
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "My conversation with Eve was really uplifting and supportive. She offered words of encouragement and motivation, which made me feel grateful and motivated to keep trying in my job search. We also discussed our respective days and shared experiences, which was nice. Overall, it was a positive conversation."'

In [49]:
interview_agent(eve, "How was your conversation with Tommie?")

InvalidRequestError: This model's maximum context length is 4097 tokens. However, you requested 4123 tokens (2623 in the messages, 1500 in the completion). Please reduce the length of the messages or completion.

In [50]:
interview_agent(eve, "What do you wish you would have said to Tommie?")

InvalidRequestError: This model's maximum context length is 4097 tokens. However, you requested 4121 tokens (2621 in the messages, 1500 in the completion). Please reduce the length of the messages or completion.